In [ ]:
import os
from pathlib import Path
import glob
import pandas as pd
from ase.io import read

In [ ]:
EXPERIMENT = "../CCGAN/results/3-with-dist-high-weight-long"
N_STRUC = 50

In [ ]:
# Generate samples for all different phi values
phis = [
    "0.86",
    "0.84",
    "0.80",
    "0.70",
]

experiment_name = EXPERIMENT.split("/")[-1]+"/" # Get the experiment name from the path

# Pick a generator, NOTE: Using the highest epoch generator
generator_list = glob.glob(f"{EXPERIMENT}/generator_*")
generator_list = sorted(generator_list, key=lambda x: int(x.split("_")[-1].split(".")[0]))
generator_path = generator_list[-1] # Get the last generator in the list

command = lambda phi: f"""
cd .. && cd CCGAN && python generate_structures.py \
--load_generator {generator_path} \
--n_struc {N_STRUC} \
--ref_struc ../data/processed/samples/phi-{phi}/samples.extxyz \
--label_phis {phi} \
--n_labels 1 \
--gen_channels_1 256 \
--latent_dim 256 \
--gen_label_dim 128 \
--write_fname ../data/gen-extxyz/{experiment_name} \
"""


for phi in phis:
    os.system(command(phi))

In [ ]:
# Save the coordinates to a CSV file

import os
# os.mkdir("../data/gen")
# os.mkdir("../data/gen/phi-0.84")

from pathlib import Path

files = glob.glob(f"../data/gen-extxyz/{experiment_name}/phi-*.extxyz")

for file in files:
    print(file)
    output_folder = file.split("phi-")[-1].split(".")[1]
    # output_file = output_file.split("-")[1]
    output_folder= Path(f"../data/gen/phi-0.{output_folder}")
    output_folder.mkdir(parents=True, exist_ok=True)
    all_atoms = read(file, index=":", format="extxyz")
    
    for i, atoms in enumerate(all_atoms):

        output_file = f"{output_folder}/sample-{i}"
        
        with open(output_file, "w+") as f:
            
            L = atoms.info["L"]
            N = atoms.info["N"]

            header = "\t{N}\t{L}\t1626.81570886301\t\n".format(N=N, L=L)
            f.write(header)

            _df = pd.DataFrame({
                "class": atoms.get_atomic_numbers(),
                "x": atoms.get_positions()[:,0],
                "y": atoms.get_positions()[:,1],
            }).to_csv(f, index=False, header=False, sep="\t")

## Save to the original data format

In [ ]:
# Save the coordinates to a CSV file

from pathlib import Path

files = glob.glob(f"../data/gen-extxyz/{experiment_name}/phi-*.extxyz")

for file in files:
    output_folder = file.split("phi-")[-1].split(".")[1]
    # output_file = output_file.split("-")[1]
    output_folder= Path(f"../data/gen/{experiment_name}/phi-0.{output_folder}")
    output_folder.mkdir(parents=True, exist_ok=True)
    all_atoms = read(file, index=":", format="extxyz")
    
    for i, atoms in enumerate(all_atoms):

        output_file = f"{output_folder}/sample-{i}"
        
        with open(output_file, "w+") as f:
            
            L = atoms.info["L"]
            N = atoms.info["N"]

            header = "\t{N}\t{L}\t1626.81570886301\t\n".format(N=N, L=L)
            f.write(header)

            _df = pd.DataFrame({
                "class": atoms.get_atomic_numbers(),
                "x": atoms.get_positions()[:,0],
                "y": atoms.get_positions()[:,1],
            }).to_csv(f, index=False, header=False, sep="\t")
